# Summary
This document corresponds to Exercise 7 of [this file](https://github.com/PerformanceEstimation/Learning-Performance-Estimation/blob/main/Exercises/Course.pdf).

The first step consists in installing [PEPit](https://pypi.org/project/PEPit/) and its dependencies.

In [ ]:
!pip install pepit

In [ ]:
from PEPit import PEP
from PEPit.operators import LipschitzOperator

n = 5
verbose = 1

# Instantiate PEP
problem = PEP()

# Declare a non expansive operator
A = problem.declare_function(LipschitzOperator, L=1.)

# Start by defining its unique optimal point xs = x_*
xs, _, _ = A.fixed_point()

# Then define the starting point x0 of the algorithm
x0 = problem.set_initial_point()

# Set the initial constraint that is the difference between x0 and xs
problem.set_initial_condition((x0 - xs) ** 2 <= 1)

# Run n steps of Halpern Iterations
x = x0
x_list = list()
x_list.append(x)
image_list = list()
for i in range(n):
    image = A.gradient(x)
    image_list.append(image)
    x = 1 / (i + 2) * x0 + (1 - 1 / (i + 2)) * image
    x_list.append(x)

image = A.gradient(x)
image_list.append(image)

# Set the performance metric to distance between xN and AxN
problem.set_performance_metric((x - image) ** 2)

# Solve the PEP
pepit_tau = problem.solve(verbose=verbose , dimension_reduction_heuristic="logdet15")


In [ ]:
iterates = list()
residuals = list()
images = list()
for i in range(n+1):
    iterates.append((x_list[i]-xs).eval()[0])
    residuals.append((image_list[i]-x_list[i]).eval()[0])
    images.append((image_list[i]).eval()[0])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(iterates, images, '.-',label='iterates')
plt.plot(iterates[0], images[0], '.',label='first')
plt.plot(iterates[n], images[n], 's',label='last')
plt.plot(iterates, iterates, '.-',label='x=y')
plt.plot(0, 0, '*',label='solution')

plt.xlabel('iterate')
plt.ylabel('residual')
plt.legend()
plt.show()